# Compiling Different Assessment Components

Combine different CSV files which contain student scores from Quizzes. Make new CSV file which contains these scores and combines them to make an overall score.
Also create histogram for each quiz and for the overall distribution of marks.
Create descriptive statistics for each quiz and for overall distribution of marks (mean, mode, median, std, etc.) 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from functools import reduce
import glob
import os
import docx

## CSV

### CSV Input

In [ ]:
path = os.getcwd()
all_files = glob.glob(path + "/*.csv")

# IMPORTANT: remove last row in csv file either with the first for loop, or setting "skipfooter = 1" further below
for file in all_files:
    df = pd.read_csv(file, index_col=None, header=0)
#REMOVING LAST ROW AS READING CSV files    
    df['Quiz Name'] = os.path.basename(file)
    df['Quiz Name'] = df['Quiz Name'].str[:6]
    df = df[:-1]
#    df = df.iloc[:-1 , :]
    df.to_csv(file)

col_list = ["Surname", "First name", "Email address", "Grade/100.0", "Quiz Name"]
df_from_each_file = (pd.read_csv(file, usecols=col_list, index_col = False, skipfooter=0, engine='python') for file in all_files)
concatenated_df = pd.concat(df_from_each_file, ignore_index = True)
concatenated_df.replace('-', np.nan, inplace = True)
concatenated_df.fillna(value = 0, inplace =True)
concatenated_df['Grade/100.0'] = concatenated_df['Grade/100.0'].astype(float)

In [ ]:
final_df = pd.pivot_table(concatenated_df, index = ["Email address", 'Surname', 'First name'], columns = ["Quiz Name"], values = "Grade/100.0")
final_df.fillna(value = 0, inplace =True)
final_df.columns = final_df.columns.get_level_values(0)
final_df.columns = [''.join(col).strip() for col in final_df.columns.values]

In [ ]:
final_df.reset_index(inplace=True)
final_df.drop(['Email address'], axis=1, inplace = True)

In [ ]:
final_df.sort_values(by='Surname', inplace = True)
final_df['Mean'] = final_df.iloc[:, [2,3,4]].mean(axis=1)
final_df['Mean'] = final_df['Mean'].round(1)

### CSV Output

In [ ]:
final_df.to_csv('Quiz_Scores.csv',index=False)

In [ ]:
for column in final_df[['Quiz_1', 'Quiz_2', 'Quiz_3', 'Mean']]:
    columnSeriesObj = final_df[column]
    figuretoplot = sns.displot(data = columnSeriesObj)
    figuretoplot.set_axis_labels(column + " scores")
    figuretoplot.savefig(column + ".png")

In [ ]:
final_df.mask(final_df == 0)[['Quiz_1','Quiz_2', 'Quiz_3', 'Mean']].describe().applymap('{:,.1f}'.format)